# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [1]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [2]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert_client.insert_client import inserir_cliente
import json

Ambiente: test


## Importando e tratando os dados

In [13]:
source = "../../data/baseTesteIXC.xlsx"
df = carregar_tabela_excel(source)

In [14]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'razaoSocial',
    'Email',
    'Celular',
    'cpf_cnpj',
    'rg_ie',
    'filial_id',
    'idCidade',
    'Endereco',
    'Complemento',
    'Numero',
    'Bairro',
    'Cep',
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Tratando CPF_CNPJ e Removendo Duplicados

In [15]:
# Garante que a coluna seja interpretada como string
df["cpf_cnpj"] = df["cpf_cnpj"].astype(str)

# Remove os caracteres '.', '-', '/' usando expressão regular
df["cpf_cnpj"] = df["cpf_cnpj"].str.replace(r"[./-]", "", regex=True)

# Remove duplicados do CPF tratando como um unico cliente
df = df.drop_duplicates(subset=["cpf_cnpj"], keep="first")

### Tratando numero de Celular

In [16]:
# Converte a coluna Celular em string
df["Celular"] = df["Celular"].astype(str)

# Remove os caracteres '(', ')' e '-' usando expressão regular
df["Celular"] = df["Celular"].str.replace(r"[()\-]", "", regex=True)

# Substitui valores 'nan' ou string vazia por '99999999999'
df["Celular"] = df["Celular"].replace(["nan", ""], "99999999999")

### Tratando rg_ie

In [17]:
## Substitui NaN por vazio no campo rg_ie
df["rg_ie"] = df["rg_ie"].fillna("")

### Tratando Complemento e Número 

In [18]:
## Substitui NaN por vazio no campo Complemento
df["Complemento"] = df["Complemento"].fillna("")


## Tratando número da casa
import re

def tratar_numero(valor):
    # Converte para string
    texto = str(valor)

    # Captura todos os dígitos (0-9) presentes no texto
    digits = re.findall(r"\d+", texto)

    if len(digits) > 0:
        # Se encontrar dígitos, junta tudo numa única string
        return "".join(digits)
    else:
        # Se não houver dígitos, retorna "SN"
        return "SN"

# Aplica a função na coluna Numero
df["Numero"] = df["Numero"].apply(tratar_numero)
df.columns

Index(['razaoSocial', 'Email', 'Celular', 'cpf_cnpj', 'rg_ie', 'filial_id',
       'idCidade', 'Endereco', 'Complemento', 'Numero', 'Bairro', 'Cep'],
      dtype='object')

### Inserção teste

In [ ]:
# inserir_cliente(
#     razao="Cliente de Teste 2",
#     cnpj_cpf="28086842000127",
#     cep="57160-000",
#     endereco="Rua ABC",
#     numero="123",
#     bairro="Bairro XPTO",
#     filial_id="35",
#     cidade="1659",
#     uf="AL",
#     telefone_celular="82999999999",
#     email="teste@example.com",
# )

## Insersão do DF

In [ ]:
# Limitando o DF
df = df.head(10)

In [53]:
# Crie as colunas adicionais (inicialmente vazias)
df["idClienteIXC"] = None
df["statusInsercao"] = None
df["logRetorno"] = None

for idx, row in df.iterrows():
    # Mapeamos as colunas do DF para os parâmetros da função inserir_cliente.
    razao = row["razaoSocial"]
    email = row["Email"]
    celular = row["Celular"]
    cnpj_cpf = row["cpf_cnpj"]
    rg_ie = row["rg_ie"]
    filial = row["filial_id"]
    cidade = row["idCidade"] 
    endereco = row["Endereco"]
    complemento = row["Complemento"]
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    
    # Chamamos a função inserir_cliente:
    response = inserir_cliente(
        razao=razao,
        cnpj_cpf=cnpj_cpf,
        cep=cep,
        endereco=endereco,
        numero=numero,
        complemento=complemento,
        bairro=bairro,
        cidade=cidade,
        filial_id=filial,
        telefone_celular=celular,
        email=email,
        ie_identidade=rg_ie,
        # Se você quiser passar outros campos com valores-padrão, inclua aqui
        # Ex.: uf="AL" etc.
    )
    
    status_code = response.status_code
    content = response.content  # bytes

    # Fazemos parse do JSON:
    # b'{"type":"success","message":"Registro inserido com sucesso!","id":"117580", ...}'
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: capturamos o "id"
        client_id = json_resp.get("id")  # Exemplo: "117580"
        
        df.at[idx, "idClienteIXC"] = client_id
        df.at[idx, "statusInsercao"] = "sucesso"
        # logRetorno pode ficar vazio ou None em caso de sucesso
        df.at[idx, "logRetorno"] = ""
    else:
        # Falha: salvamos o log completo
        df.at[idx, "idClienteIXC"] = None
        df.at[idx, "statusInsercao"] = "falha"
        # Guardamos o JSON completo (ou a chave "message", se preferir):
        df.at[idx, "logRetorno"] = json_resp

# Ao final, df terá as colunas extras preenchidas
df.head()

200 OK
b'{"type":"error","message":"Este CNPJ\\/CPF j\\u00e1 est\\u00e1 Cadastrado!<br>ID: 117164 - (ABELANIO VER\\u00cdSSIMO PEIXOTO)","atualiza_campos":[{"tipo":"r","campo":"ativo","valor":"S"},{"tipo":"i","campo":"data_cadastro","valor":""},{"tipo":"i","campo":"filial_id","valor":"35"},{"tipo":"i","campo":"latitude","valor":""},{"tipo":"i","campo":"longitude","valor":""},{"tipo":"i","campo":"id_conta","valor":""},{"tipo":"d","campo":"crm_data_vencemos","valor":""},{"tipo":"r","campo":"convert_cliente_forn","valor":""},{"tipo":"d","campo":"crm_data_perdemos","valor":""},{"tipo":"i","campo":"crm_data_sem_viabilidade","valor":""},{"tipo":"i","campo":"crm_data_sem_porta_disponivel","valor":""},{"tipo":"i","campo":"crm_data_abortamos","valor":""},{"tipo":"i","campo":"crm_data_negociando","valor":""},{"tipo":"i","campo":"crm_data_apresentando","valor":""},{"tipo":"i","campo":"crm_data_sondagem","valor":""},{"tipo":"i","campo":"crm_data_novo","valor":""}]}'
200 OK
b'{"type":"error","messag

KeyboardInterrupt: 

### Retorno

In [54]:
df

,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idClienteIXC,statusInsercao,logRetorno
0,Abelanio Veríssimo Peixoto,radiosnettelecom@gmail.com,28999537513,70380021471,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000,None,falha,"{'type': 'error', 'message': 'Este CNPJ/CPF já..."
1,Aclecia da Silva,radiosnettelecom@gmail.com,8298155632,12694798497,30234875,35,1659,Maria de Lurdes,Próximo ao mirante,SN,Centro,57830-000,None,falha,"{'type': 'error', 'message': 'Este CNPJ/CPF já..."
2,Adeilton da Mata,adeiltondamata@gmail.com,8287840401,02301397440,40259595,35,1659,Ass. Nova Esperança,próximo a casa de farinha,SN,Zona Rural,57830-000,None,falha,"{'type': 'error', 'message': 'Este CNPJ/CPF já..."
3,Adeilton da Silva,radiosnettelecom@gmail.com,82996596910,51629143472,3614522-0,35,1659,Gravatá,Lote do Sr. Del,SN,Zona Rural,57830-000,None,falha,"{'type': 'error', 'message': 'Este CNPJ/CPF já..."
4,Adeilton Teixeira de Lima,radiosnettelecom@gmail.com,82981674711,00843354402,36145220,35,1659,João Lyra / Qd: A,,24,Conj. Raimundo Nonato Lopes,57830-000,None,falha,"{'type': 'error', 'message': 'Este CNPJ/CPF já..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,Yasmim emanuele correia da Silva,radiosnettelecom@gmail.com,82 9822157025,47574821836,,35,1659,Rua marechal Floriano Peixoto,próximo ao sindicato,255,Mundo novo,57830-000,None,None,None
619,Zaquel da Costa Firmino,betania@gmail.com,82981547921,11741057477,,35,1659,Ass. Eldorado dos Carajás,mãe da Betânia,SN,Zona Rural,57830-000,None,None,None
620,Zélia Maria da Silva Oliveira,radiosnettelecom@gmail.com,82981478041,10048396427,,35,1659,platô 3/ Q: 10,,37,Conj. Raimundo Nonato Lopes,57830-000,None,None,None
621,ZENILDA DE LIMA FREIRE,radiosnettelecom@gmail.com,82982297108,04430401411,,35,1659,Assentamento Flor do Mundau,irmã do dimas,SN,Zona Rural,57830-000,None,None,None
